In [1]:
import os
import ee
import geemap
import pandas as pd
import geopandas as gpd

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [4]:
path = "_remote_sensing"

# Region of interest
region = ee.Geometry.BBox(-79.49160, -18.16247, -43.69472,  10.05915)

In [6]:
# 10 years range
time_start = "2013-01-01"
time_end = "2023-01-01" 

# Define the precipitation collection and filter the collection by time range
precipitation = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(time_start, time_end).select('precipitation')

# Function to reclassify each daily raster where values equal to 0 (no precipitation) are set to 1, and all other values are set to 0
def reclass_equal_to_zero(image):
    return image.eq(0)

# Function to calculate the yearly sum of days without precipitation
def calculate_yearly_sum(year):
    # Filter the precipitation collection for the specific year
    yearly_collection = precipitation.filter(ee.Filter.calendarRange(year, year, 'year'))
    
    # Reclassify each image in the collection
    yearly_collection_reclassified = yearly_collection.map(reclass_equal_to_zero)
    
    # Sum the reclassified images to get the yearly count of days without precipitation
    yearly_sums = yearly_collection_reclassified.sum().rename("annual days without precipitation")
    
    # Set the year as a property
    yearly_sums = yearly_sums.set('year', year)
    
    return yearly_sums

In [7]:
# Map over the years and calculate the yearly sum for each year
annual_days_without_precipitation_list = list(map(calculate_yearly_sum, range(2013, 2023)))

# Convert the list of images to an ImageCollection
annual_days_without_precipitation = ee.ImageCollection.fromImages(annual_days_without_precipitation_list)
annual_days_without_precipitation

In [8]:
# Calculate the mean of yearly sums
mean_yearly_sums = annual_days_without_precipitation.mean()
# Calculate the standard deviation of yearly sums
std_dev_yearly_sums = annual_days_without_precipitation.reduce(ee.Reducer.stdDev())

## Calculate the threshold: mean plus 2 standard deviations
mean_plus_2std = mean_yearly_sums.add(std_dev_yearly_sums.multiply(2))

In [9]:
# Function to subtract the mean plus 2 standard deviations from each yearly sums raster
def subtract_mean_plus_2std(image):
    return image.subtract(mean_plus_2std).set('system:time_start', image.get('system:time_start'))

# Function to reclassify each raster to get values above
def reclass_above(image):
    return image.gt(0)  # Create a binary mask where values greater than 0 are set to 1, and others are set to 0

In [10]:
# Number of years with precipitation above the mean plus 2 standard deviations from 2013 to 2022
anomaly_mean_plus_2std = annual_days_without_precipitation.map(subtract_mean_plus_2std) 
anomaly_mean_plus_2std = anomaly_mean_plus_2std.map(reclass_above).reduce(ee.Reducer.sum())

In [14]:
#anomaly_mean_plus_2std.getInfo()

In [11]:
# Load Amazon boundary
amazonia = gpd.read_file(os.path.join(path, 'amazonia_boundary.shp'))
amazonia = amazonia.to_crs(4326)

vis_params_mean = {'min': 0, 'max': 2, 'palette': ['0000FF', '00FF00', 'FF0000']} # Define visualization parameters
Map = geemap.Map() # Create a Map
Map.addLayer(anomaly_mean_plus_2std, vis_params_mean, '') # Add the droughts anomalies layer
Map.add_gdf(amazonia, layer_name = 'Amazon') # Add the Amazon boundary to the map
Map # Display the map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [12]:
# Clip the anomaly_mean_plus_2std raster using the region of interest
anomaly_mean_plus_2std_clip = anomaly_mean_plus_2std.clip(region).int8()

# Creates a batch task to export an Image as a raster to Drive
task = ee.batch.Export.image.toDrive(
    image = anomaly_mean_plus_2std_clip,
    description = 'drought_anomalies_mean_plus_2std_clip',
    folder = 'fulbright',
    region = region,
    scale = 5566,
    crs = 'EPSG:4326',
    maxPixels = 1e13
)
task.start()